Ethan Biegeleisen

MATG 691 Project

December 7, 2021

In [ ]:
# Imports from Course 4, Week 1, Assignment 2

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
from cnnutils import * # Code from original assignment said cnn_utils instead of cnnutils

%matplotlib inline

In [ ]:
# Imports from Course 4, Week 2, Assignment 1

import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

%matplotlib inline

### Preparing the 64 x 64 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

import glob

Images_Positive64 = []
X_Positive64 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive64 = image.load_img(filename, target_size = (64, 64))
    x_Positive64 = image.img_to_array(Image_Positive64)
    Images_Positive64.append(Image_Positive64)
    X_Positive64.append(x_Positive64)

In [ ]:
imshow(Images_Positive64[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative64 = []
X_Negative64 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative64 = image.load_img(filename, target_size = (64, 64))
    x_Negative64 = image.img_to_array(Image_Negative64)
    Images_Negative64.append(Image_Negative64)
    X_Negative64.append(x_Negative64)

In [ ]:
imshow(Images_Negative64[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive64)):
    X_Positive64[i] = X_Positive64[i]/255.0
    
for i in range(len(X_Negative64)):
    X_Negative64[i] = X_Negative64[i]/255.0

In [ ]:
# Creating labels for the image arrays
# These labels can be used for any image size

Y_Positive = np.ones(len(X_Positive64))
Y_Negative = np.zeros(len(X_Negative64))

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X64 = np.vstack((X_Positive64, X_Negative64))
Y64 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

from sklearn.model_selection import train_test_split
X_Train64, X_Test64, Y_Train64, Y_Test64 = train_test_split(X64, Y64, test_size = 0.1, random_state = 42)

### Creating the Convolutional Block Functions

In [ ]:
# Creating the convolutional block function for a standard network (no skip connection)

def convolutional_block_standard(X, f, filters, s = 2, training = True, initializer = glorot_uniform):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    # For the first component, a convolutional layer is applied to the input with filter size F1 x F1, a stride of s, and padding applied to the input 
    # This is followed by batch normalization and a ReLU activation
    
    ## Second component of main path
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    # For the second component, a convolutional layer is applied to the input with filter size F2 x F2, a stride of 1, and no padding applied to the input
    # This is followed by batch normalization and a ReLU activation
    
    ## Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)

    # For the third component, a convolutional layer is applied to the input with filter size F3 x F3, a stride of 1, and padding applied to the input
    # This is followed by batch normalization  
    
    # Final step: Pass the main path through a RELU activation
    X = Activation('relu')(X)
    
    # For the final step, the input is passed through a ReLU activation
    
    return X

In [ ]:
# Creating the convolutional block function for a residual network (skip connection)

def convolutional_block_skip(X, f, filters, s = 2, training = True, initializer = glorot_uniform):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    # For the first component, a convolutional layer is applied to the input with filter size F1 x F1, a stride of s, and padding applied to the input 
    # This is followed by batch normalization and a ReLU activation
    
    ## Second component of main path
    X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    # For the second component, a convolutional layer is applied to the input with filter size F2 x F2, a stride of 1, and no padding applied to the input
    # This is followed by batch normalization and a ReLU activation
    
    ## Third component of main path
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    
    # For the third component, a convolutional layer is applied to the input with filter size F3 x F3, a stride of 1, and padding applied to the input
    # This is followed by batch normalization    
    
    ##### SHORTCUT PATH #####
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training=training)

    # For shortcut path, a convolutional layer is applied to the initial input with filter size F3 x F3, a stride of s, and padding applied to the input 
    # This is followed by batch normalization
    
    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    # For the final step, the input that went through the third component is added with the input that went through the shorcut path
    # This is followed by a ReLU activation
    
    return X

### Using a Standard Convolutional Neural Network with 64 x 64 x 3 Images

In [ ]:
# Creating the model

def StandardModel64(input_shape = (64, 64, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Three layers of zero padding are placed around the input
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)
    
    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256

    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)
    
    # An average pooling filter is applied to the input with a filter size 2 x 2

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model64 = StandardModel64(input_shape = (64, 64, 3), classes = 1)

# Compile the model
Standard_Model64.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model64.summary()

In [ ]:
# Training the model
Standard_History64 = Standard_Model64.fit(X_Train64, Y_Train64, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History64.history['loss'])
plt.title('Standard Network Loss (64 x 64 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History64.history['accuracy'])
plt.title('Standard Network Accuracy (64 x 64 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss64, Standard_Test_Accuracy64 = Standard_Model64.evaluate(X_Test64, Y_Test64)

In [ ]:
print('Test Loss:', Standard_Test_Loss64)
print('Test Accuracy:', Standard_Test_Accuracy64)

### Using a Residual Network with 64 x 64 x 3 Images

In [ ]:
# Creating the model

def ResNet64(input_shape = (64, 64, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Three layers of zero padding are placed around the input
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2


    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)
    
    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model64 = ResNet64(input_shape = (64, 64, 3), classes = 1)

# Compile the model
ResNet_Model64.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model64.summary())

In [ ]:
# Training the model
ResNet_History64 = ResNet_Model64.fit(X_Train64, Y_Train64, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History64.history['loss'])
plt.title('Residual Network Loss (64 x 64 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History64.history['accuracy'])
plt.title('Residual Network Accuracy (64 x 64 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History64.history['loss'], color = 'c')
plt.plot(ResNet_History64.history['loss'], color = 'm')
plt.title('Network Training Loss (64 x 64 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History64.history['accuracy'], color = 'c')
plt.plot(ResNet_History64.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (64 x 64 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss64, ResNet_Test_Accuracy64 = ResNet_Model64.evaluate(X_Test64, Y_Test64)

In [ ]:
print('Test Loss:', ResNet_Test_Loss64)
print('Test Accuracy:', ResNet_Test_Accuracy64)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis64 = [Standard_Test_Loss64, ResNet_Test_Loss64]
plt.bar(MethodAxis, LossAxis64, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (64 x 64 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis64 = [Standard_Test_Accuracy64, ResNet_Test_Accuracy64]
plt.bar(MethodAxis, AccuracyAxis64, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (64 x 64 x 3)')
plt.show()

### Preparing the 32 x 32 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

Images_Positive32 = []
X_Positive32 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive32 = image.load_img(filename, target_size = (32, 32))
    x_Positive32 = image.img_to_array(Image_Positive32)
    Images_Positive32.append(Image_Positive32)
    X_Positive32.append(x_Positive32)

In [ ]:
imshow(Images_Positive32[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative32 = []
X_Negative32 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative32 = image.load_img(filename, target_size = (32, 32))
    x_Negative32 = image.img_to_array(Image_Negative32)
    Images_Negative32.append(Image_Negative32)
    X_Negative32.append(x_Negative32)

In [ ]:
imshow(Images_Negative32[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive32)):
    X_Positive32[i] = X_Positive32[i]/255.0
    
for i in range(len(X_Negative32)):
    X_Negative32[i] = X_Negative32[i]/255.0

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X32 = np.vstack((X_Positive32, X_Negative32))
Y32 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

X_Train32, X_Test32, Y_Train32, Y_Test32 = train_test_split(X32, Y32, test_size = 0.1, random_state = 42)

### Using a Standard Convolutional Neural Network with 32 x 32 x 3 Images

In [ ]:
# Creating the model

def StandardModel32(input_shape = (32, 32, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Three layers of zero padding are placed around the input
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2
    
    
    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model32 = StandardModel32(input_shape = (32, 32, 3), classes = 1)

# Compile the model
Standard_Model32.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model32.summary()

In [ ]:
# Training the model
Standard_History32 = Standard_Model32.fit(X_Train32, Y_Train32, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History32.history['loss'])
plt.title('Standard Network Loss (32 x 32 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History32.history['accuracy'])
plt.title('Standard Network Accuracy (32 x 32 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss32, Standard_Test_Accuracy32 = Standard_Model32.evaluate(X_Test32, Y_Test32)

In [ ]:
print('Test Loss:', Standard_Test_Loss32)
print('Test Accuracy:', Standard_Test_Accuracy32)

### Using a Residual Network with 32 x 32 x 3 Images

In [ ]:
# Creating the model

def ResNet32(input_shape = (32, 32, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Three layers of zero padding are placed around the input
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2
    
    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)
    
    # An average pooling filter is applied to the input with a filter size 2 x 2    

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model32 = ResNet32(input_shape = (32, 32, 3), classes = 1)

# Compile the model
ResNet_Model32.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model32.summary())

In [ ]:
# Training the model
ResNet_History32 = ResNet_Model32.fit(X_Train32, Y_Train32, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History32.history['loss'])
plt.title('Residual Network Loss (32 x 32 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History32.history['accuracy'])
plt.title('Residual Network Accuracy (32 x 32 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History32.history['loss'], color = 'c')
plt.plot(ResNet_History32.history['loss'], color = 'm')
plt.title('Network Training Loss (32 x 32 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History32.history['accuracy'], color = 'c')
plt.plot(ResNet_History32.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (32 x 32 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss32, ResNet_Test_Accuracy32 = ResNet_Model32.evaluate(X_Test32, Y_Test32)

In [ ]:
print('Test Loss:', ResNet_Test_Loss32)
print('Test Accuracy:', ResNet_Test_Accuracy32)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis32 = [Standard_Test_Loss32, ResNet_Test_Loss32]
plt.bar(MethodAxis, LossAxis32, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (32 x 32 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis32 = [Standard_Test_Accuracy32, ResNet_Test_Accuracy32]
plt.bar(MethodAxis, AccuracyAxis32, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (32 x 32 x 3)')
plt.show()

### Preparing the 16 x 16 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

Images_Positive16 = []
X_Positive16 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive16 = image.load_img(filename, target_size = (16, 16))
    x_Positive16 = image.img_to_array(Image_Positive16)
    Images_Positive16.append(Image_Positive16)
    X_Positive16.append(x_Positive16)

In [ ]:
imshow(Images_Positive16[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative16 = []
X_Negative16 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative16 = image.load_img(filename, target_size = (16, 16))
    x_Negative16 = image.img_to_array(Image_Negative16)
    Images_Negative16.append(Image_Negative16)
    X_Negative16.append(x_Negative16)

In [ ]:
imshow(Images_Negative16[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive16)):
    X_Positive16[i] = X_Positive16[i]/255.0
    
for i in range(len(X_Negative16)):
    X_Negative16[i] = X_Negative16[i]/255.0

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X16 = np.vstack((X_Positive16, X_Negative16))
Y16 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

X_Train16, X_Test16, Y_Train16, Y_Test16 = train_test_split(X16, Y16, test_size = 0.1, random_state = 42)

### Using a Standard Convolutional Neural Network with 16 x 16 x 3 Images

In [ ]:
# Creating the model

def StandardModel16(input_shape = (16, 16, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Three layers of zero padding are placed around the input
    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model16 = StandardModel16(input_shape = (16, 16, 3), classes = 1)

# Compile the model
Standard_Model16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model16.summary()

In [ ]:
# Training the model
Standard_History16 = Standard_Model16.fit(X_Train16, Y_Train16, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History16.history['loss'])
plt.title('Standard Network Loss (16 x 16 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History16.history['accuracy'])
plt.title('Standard Network Accuracy (16 x 16 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss16, Standard_Test_Accuracy16 = Standard_Model16.evaluate(X_Test16, Y_Test16)

In [ ]:
print('Test Loss:', Standard_Test_Loss16)
print('Test Accuracy:', Standard_Test_Accuracy16)

### Using a Residual Network with 16 x 16 x 3 Images

In [ ]:
# Creating the model

def ResNet16(input_shape = (16, 16, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Three layers of zero padding are placed around the input
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model16 = ResNet16(input_shape = (16, 16, 3), classes = 1)

# Compile the model
ResNet_Model16.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model16.summary())

In [ ]:
# Training the model
ResNet_History16 = ResNet_Model16.fit(X_Train16, Y_Train16, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History16.history['loss'])
plt.title('Residual Network Loss (16 x 16 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History16.history['accuracy'])
plt.title('Residual Network Accuracy (16 x 16 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History16.history['loss'], color = 'c')
plt.plot(ResNet_History16.history['loss'], color = 'm')
plt.title('Network Training Loss (16 x 16 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History16.history['accuracy'], color = 'c')
plt.plot(ResNet_History16.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (16 x 16 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss16, ResNet_Test_Accuracy16 = ResNet_Model16.evaluate(X_Test16, Y_Test16)

In [ ]:
print('Test Loss:', ResNet_Test_Loss16)
print('Test Accuracy:', ResNet_Test_Accuracy16)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis16 = [Standard_Test_Loss16, ResNet_Test_Loss16]
plt.bar(MethodAxis, LossAxis16, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (16 x 16 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis16 = [Standard_Test_Accuracy16, ResNet_Test_Accuracy16]
plt.bar(MethodAxis, AccuracyAxis16, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (16 x 16 x 3)')
plt.show()

### Preparing the 8 x 8 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

Images_Positive8 = []
X_Positive8 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive8 = image.load_img(filename, target_size = (8, 8))
    x_Positive8 = image.img_to_array(Image_Positive8)
    Images_Positive8.append(Image_Positive8)
    X_Positive8.append(x_Positive8)

In [ ]:
imshow(Images_Positive8[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative8 = []
X_Negative8 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative8 = image.load_img(filename, target_size = (8, 8))
    x_Negative8 = image.img_to_array(Image_Negative8)
    Images_Negative8.append(Image_Negative8)
    X_Negative8.append(x_Negative8)

In [ ]:
imshow(Images_Negative8[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive8)):
    X_Positive8[i] = X_Positive8[i]/255.0
    
for i in range(len(X_Negative8)):
    X_Negative8[i] = X_Negative8[i]/255.0

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X8 = np.vstack((X_Positive8, X_Negative8))
Y8 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

X_Train8, X_Test8, Y_Train8, Y_Test8 = train_test_split(X8, Y8, test_size = 0.1, random_state = 42)

### Using a Standard Convolutional Neural Network with 8 x 8 x 3 Images

In [ ]:
# Creating the model

def StandardModel8(input_shape = (8, 8, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Zero-Padding
    X = ZeroPadding2D((4, 4))(X_input)
    
    # Four layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)
    
    # An average pooling filter is applied to the input with a filter size 2 x 2

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model8 = StandardModel8(input_shape = (8, 8, 3), classes = 1)

# Compile the model
Standard_Model8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model8.summary()

In [ ]:
# Training the model
Standard_History8 = Standard_Model8.fit(X_Train8, Y_Train8, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History8.history['loss'])
plt.title('Standard Network Loss (8 x 8 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History8.history['accuracy'])
plt.title('Standard Network Accuracy (8 x 8 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss8, Standard_Test_Accuracy8 = Standard_Model8.evaluate(X_Test8, Y_Test8)

In [ ]:
print('Test Loss:', Standard_Test_Loss8)
print('Test Accuracy:', Standard_Test_Accuracy8)

### Using a Residual Network with 8 x 8 x 3 Images

In [ ]:
# Creating the model

def ResNet8(input_shape = (8, 8, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((4, 4))(X_input)
    
    # Four layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2
    
    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model8 = ResNet8(input_shape = (8, 8, 3), classes = 1)

# Compile the model
ResNet_Model8.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model8.summary())

In [ ]:
# Training the model
ResNet_History8 = ResNet_Model8.fit(X_Train8, Y_Train8, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History8.history['loss'])
plt.title('Residual Network Loss (8 x 8 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History8.history['accuracy'])
plt.title('Residual Network Accuracy (8 x 8 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History8.history['loss'], color = 'c')
plt.plot(ResNet_History8.history['loss'], color = 'm')
plt.title('Network Training Loss (8 x 8 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History8.history['accuracy'], color = 'c')
plt.plot(ResNet_History8.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (8 x 8 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss8, ResNet_Test_Accuracy8 = ResNet_Model8.evaluate(X_Test8, Y_Test8)

In [ ]:
print('Test Loss:', ResNet_Test_Loss8)
print('Test Accuracy:', ResNet_Test_Accuracy8)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis8 = [Standard_Test_Loss8, ResNet_Test_Loss8]
plt.bar(MethodAxis, LossAxis8, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (8 x 8 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis8 = [Standard_Test_Accuracy8, ResNet_Test_Accuracy8]
plt.bar(MethodAxis, AccuracyAxis8, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (8 x 8 x 3)')
plt.show()

### Preparing the 4 x 4 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

Images_Positive4 = []
X_Positive4 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive4 = image.load_img(filename, target_size = (4, 4))
    x_Positive4 = image.img_to_array(Image_Positive4)
    Images_Positive4.append(Image_Positive4)
    X_Positive4.append(x_Positive4)

In [ ]:
imshow(Images_Positive4[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative4 = []
X_Negative4 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative4 = image.load_img(filename, target_size = (4, 4))
    x_Negative4 = image.img_to_array(Image_Negative4)
    Images_Negative4.append(Image_Negative4)
    X_Negative4.append(x_Negative4)

In [ ]:
imshow(Images_Negative4[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive4)):
    X_Positive4[i] = X_Positive4[i]/255.0
    
for i in range(len(X_Negative4)):
    X_Negative4[i] = X_Negative4[i]/255.0

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X4 = np.vstack((X_Positive4, X_Negative4))
Y4 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

X_Train4, X_Test4, Y_Train4, Y_Test4 = train_test_split(X4, Y4, test_size = 0.1, random_state = 42)

### Using a Standard Convolutional Neural Network with 4 x 4 x 3 Images

In [ ]:
# Creating the model

def StandardModel4(input_shape = (4, 4, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((6, 6))(X_input)
    
    # Six layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)
    
    # An average pooling filter is applied to the input with a filter size 2 x 2

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model4 = StandardModel4(input_shape = (4, 4, 3), classes = 1)

# Compile the model
Standard_Model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model4.summary()

In [ ]:
# Training the model
Standard_History4 = Standard_Model4.fit(X_Train4, Y_Train4, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History4.history['loss'])
plt.title('Standard Network Loss (4 x 4 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History4.history['accuracy'])
plt.title('Standard Network Accuracy (4 x 4 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss4, Standard_Test_Accuracy4 = Standard_Model4.evaluate(X_Test4, Y_Test4)

In [ ]:
print('Test Loss:', Standard_Test_Loss4)
print('Test Accuracy:', Standard_Test_Accuracy4)

### Using a Residual Network with 4 x 4 x 3 Images

In [ ]:
# Creating the model

def ResNet4(input_shape = (4, 4, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((6, 6))(X_input)
    
    # Six layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation

    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model4 = ResNet4(input_shape = (4, 4, 3), classes = 1)

# Compile the model
ResNet_Model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model4.summary())

In [ ]:
# Training the model
ResNet_History4 = ResNet_Model4.fit(X_Train4, Y_Train4, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History4.history['loss'])
plt.title('Residual Network Loss (4 x 4 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History4.history['accuracy'])
plt.title('Residual Network Accuracy (4 x 4 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History4.history['loss'], color = 'c')
plt.plot(ResNet_History4.history['loss'], color = 'm')
plt.title('Network Training Loss (4 x 4 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History4.history['accuracy'], color = 'c')
plt.plot(ResNet_History4.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (4 x 4 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss4, ResNet_Test_Accuracy4 = ResNet_Model4.evaluate(X_Test4, Y_Test4)

In [ ]:
print('Test Loss:', ResNet_Test_Loss4)
print('Test Accuracy:', ResNet_Test_Accuracy4)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis4 = [Standard_Test_Loss4, ResNet_Test_Loss4]
plt.bar(MethodAxis, LossAxis4, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (4 x 4 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis4 = [Standard_Test_Accuracy4, ResNet_Test_Accuracy4]
plt.bar(MethodAxis, AccuracyAxis4, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (4 x 4 x 3)')
plt.show()

### Preparing the 2 x 2 x 3 Images

In [ ]:
# Importing every image from the positive dataset and turning each image into an array

Images_Positive2 = []
X_Positive2 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/COVID-19 (944 Images)/*.png'):
    Image_Positive2 = image.load_img(filename, target_size = (2, 2))
    x_Positive2 = image.img_to_array(Image_Positive2)
    Images_Positive2.append(Image_Positive2)
    X_Positive2.append(x_Positive2)

In [ ]:
imshow(Images_Positive2[851])

In [ ]:
# Importing every image from the negative dataset and turning each image into an array

Images_Negative2 = []
X_Negative2 = []
for filename in glob.glob('Documents/Manhattan College/Fall 2021/MATG 691/MATG 691 Project/Aria Data/COVID-19_Lung_CT_Scans/Non-COVID-19/*.png'):
    Image_Negative2 = image.load_img(filename, target_size = (2, 2))
    x_Negative2 = image.img_to_array(Image_Negative2)
    Images_Negative2.append(Image_Negative2)
    X_Negative2.append(x_Negative2)

In [ ]:
imshow(Images_Negative2[821])

In [ ]:
# Standardizing the image arrays

for i in range(len(X_Positive2)):
    X_Positive2[i] = X_Positive2[i]/255.0
    
for i in range(len(X_Negative2)):
    X_Negative2[i] = X_Negative2[i]/255.0

In [ ]:
# Creating X and Y by stacking the positive and negative image arrays

X2 = np.vstack((X_Positive2, X_Negative2))
Y2 = np.hstack((Y_Positive, Y_Negative))

In [ ]:
# Splitting X and Y into train and test sets

X_Train2, X_Test2, Y_Train2, Y_Test2 = train_test_split(X2, Y2, test_size = 0.1, random_state = 42)

### Using a Standard Convolutional Neural Network with 2 x 2 x 3 Images

In [ ]:
# Creating the model

def StandardModel2(input_shape = (2, 2, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (no skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((7, 7))(X_input)
    
    # Seven layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_standard(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the standard convolutional block function is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
Standard_Model2 = StandardModel2(input_shape = (2, 2, 3), classes = 1)

# Compile the model
Standard_Model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
Standard_Model2.summary()

In [ ]:
# Training the model
Standard_History2 = Standard_Model2.fit(X_Train2, Y_Train2, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History2.history['loss'])
plt.title('Standard Network Loss (2 x 2 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History2.history['accuracy'])
plt.title('Standard Network Accuracy (2 x 2 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Testing the model
Standard_Test_Loss2, Standard_Test_Accuracy2 = Standard_Model2.evaluate(X_Test2, Y_Test2)

In [ ]:
print('Test Loss:', Standard_Test_Loss2)
print('Test Accuracy:', Standard_Test_Accuracy2)

### Using a Residual Network with 2 x 2 x 3 Images

In [ ]:
# Creating the model

def ResNet2(input_shape = (2, 2, 3), classes = 1):
    """
    Stage-wise implementation of the architecture of the standard convolutional neural network for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK (skip connection) -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((7, 7))(X_input)
    
    # Seven layers of zero padding are placed around the input
    # The amount of padding is increased to compensate for the smaller input array
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # For Stage 1, a convolutional layer is applied to the input with filter size 64 x 64 and a stride of 2
    # This is followed by batch normalization and a ReLU activation
    # Lastly, a max pooling layer is applied with filter size 3 x 3 and a stride of 2

    # Stage 2
    X = convolutional_block_skip(X, f = 3, filters = [64, 64, 256], s = 1)

    # For Stage 2, the convolutional block function with a skip connection is applied to the input with filter sizes of 64 x 64, 64 x 64, and 256 x 256
    
    ## AVGPOOL
    X = AveragePooling2D(pool_size = (2, 2))(X)

    # An average pooling filter is applied to the input with a filter size 2 x 2
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # The final section involves flattening the input into a vector
    # The vector is then passed through a dense neural network layer with sigmoid activation
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [ ]:
ResNet_Model2 = ResNet2(input_shape = (2, 2, 3), classes = 1)

# Compile the model
ResNet_Model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the summary of the model
print(ResNet_Model2.summary())

In [ ]:
# Training the model
ResNet_History2 = ResNet_Model2.fit(X_Train2, Y_Train2, epochs = 200, batch_size = 8)

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History2.history['loss'])
plt.title('Residual Network Loss (2 x 2 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(ResNet_History2.history['accuracy'])
plt.title('Residual Network Accuracy (2 x 2 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training'], loc='upper left')
plt.show()

In [ ]:
# Plotting the loss during training

# Information about plotting loss: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History2.history['loss'], color = 'c')
plt.plot(ResNet_History2.history['loss'], color = 'm')
plt.title('Network Training Loss (2 x 2 x 3)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='upper right')
plt.show()

In [ ]:
# Plotting the accuracy during training

# Information about plotting accuracy: 
# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

plt.plot(Standard_History2.history['accuracy'], color = 'c')
plt.plot(ResNet_History2.history['accuracy'], color = 'm')
plt.title('Network Training Accuracy (2 x 2 x 3)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Standard', 'Residual'], loc='lower right')
plt.show()

In [ ]:
# Testing the model
ResNet_Test_Loss2, ResNet_Test_Accuracy2 = ResNet_Model2.evaluate(X_Test2, Y_Test2)

In [ ]:
print('Test Loss:', ResNet_Test_Loss2)
print('Test Accuracy:', ResNet_Test_Accuracy2)

In [ ]:
# Plotting the Test Loss

MethodAxis = ['Standard Network', 'Residual Network']
LossAxis2 = [Standard_Test_Loss2, ResNet_Test_Loss2]
plt.bar(MethodAxis, LossAxis2, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Loss')
plt.title('Test Loss with Each Neural Network (2 x 2 x 3)')
plt.show()

In [ ]:
# Plotting the Test Accuracy

AccuracyAxis2 = [Standard_Test_Accuracy2, ResNet_Test_Accuracy2]
plt.bar(MethodAxis, AccuracyAxis2, width = 0.5)
plt.xlabel('Neural Network')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy with Each Neural Network (2 x 2 x 3)')
plt.show()

### Comparing the Test Loss and Test Accuracy of each Network using each Image Size

In [ ]:
# Plotting the Test Loss of each Standard Convolutional Neural Network

SizeAxis = ['64', '32', '16', '8', '4', '2']
StandardLossAxisAll = [Standard_Test_Loss64, Standard_Test_Loss32, Standard_Test_Loss16, Standard_Test_Loss8, Standard_Test_Loss4, Standard_Test_Loss2]
plt.bar(SizeAxis, StandardLossAxisAll, width = 0.5)
plt.xlabel('Image Size')
plt.ylabel('Test Loss')
plt.title('Standard Network Test Loss')
plt.show()

In [ ]:
# Plotting the Test Accuracy of each Sequential Neural Network

StandardAccuracyAxisAll = [Standard_Test_Accuracy64, Standard_Test_Accuracy32, Standard_Test_Accuracy16, Standard_Test_Accuracy8, Standard_Test_Accuracy4, Standard_Test_Accuracy2]
plt.bar(SizeAxis, StandardAccuracyAxisAll, width = 0.5)
plt.xlabel('Image Size')
plt.ylabel('Test Accuracy')
plt.title('Standard Network Test Accuracy')
plt.show()

In [ ]:
# Plotting the Test Loss of each Residual Network

ResNetLossAxisAll = [ResNet_Test_Loss64, ResNet_Test_Loss32, ResNet_Test_Loss16, ResNet_Test_Loss8, ResNet_Test_Loss4, ResNet_Test_Loss2]
plt.bar(SizeAxis, ResNetLossAxisAll, width = 0.5)
plt.xlabel('Image Size')
plt.ylabel('Test Loss')
plt.title('Residual Network Test Loss')
plt.show()

In [ ]:
# Plotting the Test Accuracy of each Residual Network

ResNetAccuracyAxisAll = [ResNet_Test_Accuracy64, ResNet_Test_Accuracy32, ResNet_Test_Accuracy16, ResNet_Test_Accuracy8, ResNet_Test_Accuracy4, ResNet_Test_Accuracy2]
plt.bar(SizeAxis, ResNetAccuracyAxisAll, width = 0.5)
plt.xlabel('Image Size')
plt.ylabel('Test Accuracy')
plt.title('Residual Network Test Accuracy')
plt.show()

In [ ]:
# Plotting the Test Loss of every Convolutional Neural Network

SizeAxisLabels = ['64', '32', '16', '8', '4', '2']
SizeAxis = np.arange(len(SizeAxisLabels))
StandardLossAxisAll = [Standard_Test_Loss64, Standard_Test_Loss32, Standard_Test_Loss16, Standard_Test_Loss8, Standard_Test_Loss4, Standard_Test_Loss2]
ResNetLossAxisAll = [ResNet_Test_Loss64, ResNet_Test_Loss32, ResNet_Test_Loss16, ResNet_Test_Loss8, ResNet_Test_Loss4, ResNet_Test_Loss2]

fig, ax = plt.subplots()
ax.bar(SizeAxis - 0.15, StandardLossAxisAll, width = 0.3, color = 'c', label = 'Standard')
ax.bar(SizeAxis + 0.15, ResNetLossAxisAll, width = 0.3, color = 'm', label = 'Residual')
ax.set_xlabel('Image Size')
ax.set_ylabel('Test Loss')
ax.set_title('Network Test Loss')
ax.set_xticks(SizeAxis)
ax.set_xticklabels(SizeAxisLabels)
ax.legend(loc = 'upper left')
fig.tight_layout()
plt.show()

In [ ]:
# Plotting the Test Accuracy of every Convolutional Neural Network

SizeAxisLabels = ['64', '32', '16', '8', '4', '2']
SizeAxis = np.arange(len(SizeAxisLabels))
StandardAccuracyAxisAll = [Standard_Test_Accuracy64, Standard_Test_Accuracy32, Standard_Test_Accuracy16, Standard_Test_Accuracy8, Standard_Test_Accuracy4, Standard_Test_Accuracy2]
ResNetAccuracyAxisAll = [ResNet_Test_Accuracy64, ResNet_Test_Accuracy32, ResNet_Test_Accuracy16, ResNet_Test_Accuracy8, ResNet_Test_Accuracy4, ResNet_Test_Accuracy2]

fig, ax = plt.subplots()
ax.bar(SizeAxis - 0.15, StandardAccuracyAxisAll, width = 0.3, color = 'c', label = 'Standard')
ax.bar(SizeAxis + 0.15, ResNetAccuracyAxisAll, width = 0.3, color = 'm', label = 'Residual')
ax.set_xlabel('Image Size')
ax.set_ylabel('Test Accuracy')
ax.set_title('Network Test Accuracy')
ax.set_xticks(SizeAxis)
ax.set_xticklabels(SizeAxisLabels)
ax.legend(loc = 'lower right')
fig.tight_layout()
plt.show()